In [ ]:
pip install ftfy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.7 MB/s eta 0:00:00


In [ ]:
pip install nltk

In [ ]:
pip install Unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 9.7 MB/s eta 0:00:00


In [ ]:
pip install SpaCy

In [ ]:
import re
from ftfy import fix_text
import spacy
import json
from unidecode import unidecode
spacy_model = spacy.load('en_core_web_sm')

# open write and read json file
with open('gg2013.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# loop to fix text{} in json data
for data_text in data:
    original_text = data_text['text']
    #remove URLs regular expressions
    remove_urls= re.sub(r'https?://\S+|www\.\S+', '', original_text)
    #Fix encoding issues
    fixed_text = fix_text(remove_urls)
    #Standardize special characters / emojis to Unicode
    new_text = unidecode(fixed_text)
    #clean white space
    white_space = " ".join(new_text.split())
    #keep tabs/newline characters
    tab_newline = re.sub(' +', ' ', white_space)
    # lowcase letters
    low_letter = tab_newline.lower()
    #remove hashtags
    hash_tag = re.sub(r'#', '', low_letter)
    #After pre-processing, information going back to orginal json data
    data_text['text'] = hash_tag

with open('gg2013_output.json', 'w', encoding='utf-8') as output_file:
    json.dump(data, output_file, indent=6)



###Later use
    #spacy model to syntactic parsing
    # spacy_output = spacy_model(low_letter)
    # for token in spacy_output:
    #     print(
    #     token.text,
    #     token.dep_,
    #     token.head.text,
    #     token.head.pos_,
    #     [child for child in token.children]
    #     )
